In [ ]:
import os    as os

import numpy as np

import xarray as xr

import pandas as pd


In [ ]:
def days_per_year(year):
    if (year % 4) == 0:
        if (year % 100) == 0:
            if (year %  400) == 0:
                days_per_year = 366
            else:
                days_per_year = 365
        else:
            days_per_year = 366
    else:
        days_per_year = 365
    return days_per_year

In [ ]:
working_directory  = "./"

loca2_inv_file     = "./LOCA2_Model_Member_Available_List.csv"

globus_root_dir    = "/data/GLOBUS/wjc"

input_root_dir   = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Original_CONUS/"
output_root_dir  = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Climate_CONUS/Monthly/"
variables          = ["pr"]

scenarios          = ["historical", 
                      "ssp245", 
                      "ssp370", 
                      "ssp585"]


In [ ]:
loca2_ensembles_list = pd.read_csv(filepath_or_buffer = loca2_inv_file)
models  = loca2_ensembles_list[ "Model"].values
members = loca2_ensembles_list["Member"].values

display(loca2_ensembles_list)



In [ ]:
meta_script_name = "./CDO_PREC_FIX.csh"
f0 = open(file     = meta_script_name, 
         mode      = 'w', 
         encoding  = "utf-8")


for m in range(len(models)):
    
    print("# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$", file = f0)
    
    model  = models[m]
    member = members[m]
    
    meta_deleteme = True
    
    for scenario in scenarios:
        
        print("# ================================================", file = f0)
        
        inventory = loca2_ensembles_list.iloc[m].loc[scenario]
        
        input_file = input_root_dir                              + \
                     scenario + "/"                              + \
                     "LOCA2-CONUS___"                            + \
                     loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                     loca2_ensembles_list.at[m,"Member"] + "___" + \
                     scenario + ".nc"
        
        interim_daily_files  = output_root_dir                             + \
                               scenario + "/"                              + \
                               "LOCA2-CONUS-DAILY___*___"                  + \
                               loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                               loca2_ensembles_list.at[m,"Member"] + "___" + \
                               scenario + ".nc"
        
        cod_climate_files = output_root_dir                             + \
                                scenario + "/"                              + \
                                "LOCA2-CONUS-MONTHLY___*___"                 + \
                                loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                                loca2_ensembles_list.at[m,"Member"] + "___" + \
                                scenario + ".nc"
        
        final_file = output_root_dir                             + \
                     scenario + "/"                              + \
                     "LOCA2-CONUS-MONTHLY___"                     + \
                     loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                     loca2_ensembles_list.at[m,"Member"] + "___" + \
                     scenario + ".nc"
        
        
        if (inventory != "---"):
            
            print("# ",model,member,scenario,inventory, file = f0)
            
            if ("P" in inventory):
                
                    print("# ------------------------------------------------", file = f0)
                    
                    variable    = "pr"
                    
                    interim_daily_file =  output_root_dir           + \
                        scenario                            +  "/"  + \
                        "LOCA2-CONUS-DAILY___" + variable   + "___" + \
                        loca2_ensembles_list.at[m,"Model"]  +  "."  + \
                        loca2_ensembles_list.at[m,"Member"] + "___" + \
                        scenario                            + ".nc"
                    command_isolate   = "ncks -v " +  variable   + " " + input_file + " " + interim_daily_file
                    print(command_isolate,   file = f0)


                    print("#  . . . . . . . . . . . . . . . . . . . . . . . .", file = f0)
                    cod_climate_file =  output_root_dir         +         \
                        scenario                                +  "/"  + \
                        "LOCA2-CONUS-ANNUAL_SUM___" + variable  + "___" + \
                        loca2_ensembles_list.at[m,"Model"]      +  "."  + \
                        loca2_ensembles_list.at[m,"Member"]     + "___" + \
                        scenario                                + ".nc"
                    cdo_command       = "nohup cdo --no_history -f nc4 -z zip_8 yearsum "
                    command_aggregate = cdo_command + interim_daily_file + " " + cod_climate_file                    
                    print(command_aggregate, file = f0)  
                    print('nohup ncatted -h -O -a  cell_methods,pr,m,c,"time: sum within days  time: sum over months" ' + cod_climate_file, file=f0)
                
        print("#  ...............................................", file = f0)

        command_clean    = "rm -v  " + interim_daily_files
        print(command_clean,    file = f0)
        
print("# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$", file = f0)

f0.close()
os.system("chmod a+x " + meta_script_name)            

        



In [ ]:
print("end!")